Notebook de base para armazenar as minhas anotações dos cursos.
Tomarei como base os microdados do enem de 2019. Disponibilizados no [link](https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem) . Consultado em 07/01/2021
